In [1]:
import pandas as pd
import os

In [2]:
csvfile1 = 'Resources/Clean_Hawaii_Measurements.csv'

In [3]:
csvfile2 = "Resources/Clean_Hawaii_stations.csv"

In [4]:
CHM_df = pd.read_csv(csvfile1,dtype=object)
CHS_df =pd.read_csv(csvfile2,dtype=object)

In [5]:

#CHM_df['date'] = pd.to_datetime(CHM_df['date'])
CHM_df.dtypes

station    object
date       object
prcp       object
tobs       object
dtype: object

In [6]:
CHM_df.head()


,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.0,63
2,USC00519397,2010-01-03,0.0,74
3,USC00519397,2010-01-04,0.0,76
4,USC00519397,2010-01-07,0.06,70


In [7]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column,Integer, String, Numeric, Text, Float, Date


In [60]:
!rm hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [62]:
conn = engine.connect()
CHS_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [63]:
Base = declarative_base()

#Base.Measurement.drop(engine)

class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key = True)
    station = Column(String)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
    def __repr__(self):
        return f"i{self.id}, name={self.name}"

class Station(Base):
    __tablename__ = 'station'
    station = Column(String , primary_key = True)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    def __repr__(self):
        return f"i{self.id}, name={self.name}"

In [64]:
Base.metadata.create_all(engine)

In [65]:
measurement_data = CHM_df.to_dict(orient='records')
station_data = CHS_df.to_dict(orient='records')

In [66]:
measurement_data[0]

{'date': '2010-01-01', 'prcp': '0.08', 'station': 'USC00519397', 'tobs': '65'}

In [67]:
Metadata = MetaData(bind=engine)
Metadata.reflect()

In [68]:
measurement_table = sqlalchemy.Table('measurement',Metadata,autoload=True)
station_table = sqlalchemy.Table('station',Metadata,autoload=True)

In [73]:
conn.execute(measurement_table.delete())
conn.execute(station_table.delete())

In [74]:
conn.execute(measurement_table.insert(),measurement_data)
conn.execute(station_table.insert(),station_data)

In [75]:
conn.execute('select * from measurement limit 5;').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [77]:

conn.execute('select * from station limit 5;').fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [78]:
import pandas as pd

In [80]:
station_df = pd.read_sql('select * from station;',conn)

In [81]:
station_df

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [85]:
measurement_df = pd.read_sql('select date,prcp from measurement;',conn)

In [86]:
measurement_df.head()

,date,prcp
0,2010-01-01,0.08
1,2010-01-02,0.00
2,2010-01-03,0.00
3,2010-01-04,0.00
4,2010-01-07,0.06
